In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score

# 1. 데이터 로드 및 병합
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
all_data = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)



In [2]:
# ---------------------------------------------------------
# Family Survival Rate 계산
# ---------------------------------------------------------

# (1) 성(Last Name) 추출
all_data['Last_Name'] = all_data['Name'].apply(lambda x: str(x).split(',')[0])

# (2) 초기값 설정 (0.5 = 정보 없음 / 중립)
DEFAULT_SURVIVAL_VALUE = 0.5
all_data['Family_Survival'] = DEFAULT_SURVIVAL_VALUE

# (3) 그룹핑: "성"과 "요금"이 같거나, "티켓"이 같은 사람들을 가족으로 묶음
# 이것은 입자들을 'Cluster'로 묶는 Renormalization Group과 비슷합니다.
for grp, grp_df in all_data.groupby(['Last_Name', 'Fare']):
    if (len(grp_df) != 1):
        # 가족 중 누군가 생존/사망 정보가 있다면 가져옴
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                all_data.loc[all_data['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin == 0.0):
                all_data.loc[all_data['PassengerId'] == passID, 'Family_Survival'] = 0

# (4) 티켓 번호로 한 번 더 보정 (성은 달라도 티켓이 같으면 일행)
for _, grp_df in all_data.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival'] == 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    all_data.loc[all_data['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin == 0.0):
                    all_data.loc[all_data['PassengerId'] == passID, 'Family_Survival'] = 0
                   

In [3]:
# ---------------------------------------------------------
# 기존 전처리 (간소화)
# ---------------------------------------------------------
all_data['Title'] = all_data['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
all_data['Title'] = all_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                               'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
all_data['Title'] = all_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
all_data['Title'] = all_data['Title'].replace('Mme', 'Mrs')

# Title별 나이 채우기
title_ages = all_data.groupby('Title')['Age'].transform('median')
all_data['Age'] = all_data['Age'].fillna(title_ages)

# Fare 결측치 채우기
all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].median())
# Fare Binning (수치형을 구간화하여 노이즈 감소)
all_data['FareBin'] = pd.qcut(all_data['Fare'], 5, labels=False) 

# 불필요 컬럼 제거 (Family_Survival이 있으므로 Name, Ticket 등 제거)
all_data = all_data.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId', 'SibSp', 'Parch', 'Last_Name'])


In [4]:
# ---------------------------------------------------------
# 3. 모델링
# ---------------------------------------------------------
train_len = len(train)
X = all_data[:train_len].drop(columns=['Survived'])
y = all_data[:train_len]['Survived']
X_test = all_data[train_len:].drop(columns=['Survived'])

# Family_Survival은 이미 강력한 숫자(0, 0.5, 1)이므로 스케일링 불필요
# 나머지 범주형 변수만 인코딩
cat_cols = ['Pclass', 'Sex', 'Embarked', 'Title']
num_cols = ['Age', 'Fare', 'Family_Survival', 'FareBin']

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("cat", categorical_pipe, cat_cols),
        ("num", "passthrough", num_cols) # 수치형은 그대로 통과
    ]
)


In [6]:
# KNN이나 RF가 이 변수와 궁합이 좋습니다. 
# 여기서는 앙상블 효과를 위해 보편적인 RandomForest를 씁니다.
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10, 
    min_samples_split=5, 
    random_state=42
)

clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model),
])


In [9]:
## 고정된 CV (StratifiedKFold: 클래스 비율 유지)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(clf, X, y, cv=cv, scoring="accuracy")
print("CV accuracy: %.4f ± %.4f" % (scores.mean(), scores.std()))

CV accuracy: 0.8541 ± 0.0164


In [11]:
# 학습 및 제출
clf.fit(X, y)
pred = clf.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": pred.astype(int)
})
submission.to_csv("submission.csv", index=False)